# Orientation

In this 10 minute tutorial, we'll **create**, **explore** and **customize** the 3 main elements of an API Logic Server:
1. the JSON:**API**
1. Business **Logic**
1. A Basic Web App

You are in a Jupyter Notebook.  It has a **Table of Contents** (left), and some **Terminal** windows (at bottom)

Installation has already been completed for both [Python](#Installation) and [ApiLogicServer](https://github.com/valhuber/ApiLogicServer#installation).

ApiLogicServer installs with a [Northwind sample database](Sample-Database), which we'll be using in this Tutorial.

To advance, click the down arrow, or select a section at left.

# Create Server

To create the Sample App
1. Open a Terminal: **File > New > Terminal** (you may wish to split the window to the right)
2. Run the ApiLogicServer CLI:

```
ApiLogicServer run
```
> Use the default ```db_url``` parameter for the supplied [sample database](../../wiki/Sample-Database).
Later, specify a [SQLAlchemy url](https://docs.sqlalchemy.org/en/14/core/engines.html)
to use your own database.

This creates a complete, executable ApiLogicServer in a directory called ```api_logic_server```.  Spend a moment to explore the created project - use an IDE (PyCharm, VSCode, etc) or an Editor (Atom, Text Wrangler, etc):

<figure><img src="https://github.com/valhuber/ApiLogicServer/blob/main/images/generated-project.png?raw=true"></figure>

# 1. Explore the API

The ```run``` command not only _created_ the project, but it _ran_ ```python api_logic_server/api_logic_server_run.pty``` to start the server.  Let's explore it.

## Get: Open API
Run the Open API (aka Swagger) interface in your [Browser](https://notebooks.gesis.org/binder/jupyter/user/valhuber-apilogicservertutorial-6easlb4a/proxy/5000/)  (note: locally, it would be localhost:5000), and:
1. Click **Customer**
2. Click **Get**
3. Click **Try it out**
4. Click **Execute**


> **Under investigation: how to open Browser.**  From [this page](https://discourse.jupyter.org/t/webbrowser-module/2701/2), this suggestion just outputs False.

```
import webbrowser
webbrowser.open(‘https://Google.com’)
```

In many projects, User Interface development is blocked by waiting on API creation.

> **Key Takeaway:** ApiLogicServer unblocks UI development with instant API creation.

#### Include Joins, Fields
JSON:API enables clients to control the content of returned data.  That's important, since it can reduce network traffic instead of requiring calls on multiple APIs that don't return quite the correct data.  As shown in the diagram below:
1. Specify ```include``` as ```OrderList``` - see data related per foreign keys
* This integrates well with client-side "stiching" in frameworks such as React.
2. Specify ```filter[Country]``` as **Mexico** - filtering

<figure><img src="https://github.com/valhuber/ApiLogicServer/raw/main/images/tutorial/explore-api.png"></figure>

#### Filtering, Pagination
Explore support for these required services, including filtering.


## Customize: Add New Endpoints
Your API is derived from the database, but it is not restricted to that.  You can customize your API, e.g., add new endpoints.

For example - an instant API for a database is all well and good - but can we build the "hello world" so desperately sought by
enterprises all over the world?  We can.  Examine the following code in ```api_logic_server/api_logic_server_run.py```:

```
@app.route('/hello_world')
def hello_world():  # test it with: http://localhost:5000/hello_world?user=ApiLogicServer
    """
    This is inserted to illustrate that APIs not limited to database objects, but are extensible.

    See: https://github.com/thomaxxl/safrs/wiki/Customization
    """
    user = request.args.get('user')
    return jsonify({"result": f'hello, {user}'})
```

Edit ```api_logic_server/api_logic_server_run.py``` to change the return (e.g., change _hello_ to _hi there_).
To see it run:
1. Stop the server (Ctl-c)
2. Restart it
```
cd api_logic_server
python api_logic_server_run.py
```
3. Paste this into your browser: [http://localhost:5000/hello_world?user=ApiLogicServer](http://localhost:5000/hello_world?user=ApiLogicServer)

## Patch: Logic Enabled Updates
Your API also includes services for insert, update and delete. We can use curl to test the update 
```bash
curl -vX PATCH "http://localhost:5000/Customer/ALFKI/" -H  "accept: application/vnd.api+json" -H  "Content-Type: application/json"  -d '
{
  "data": {
     "attributes": {
        "CreditLimit": "100"
     },
  "type": "Customer",
  "id": "ALFKI"
}}'
```

This update request violates our predefined rules so it will return an error, "`balance (1016.0000000000) exceeds credit (100)`" :
```json
{
    "errors": [
        {
            "title": "balance (1016.0000000000) exceeds credit (100)",
            "detail": {
                "model": "Customer",
                "error_attributes": []
            },
            "code": 2001
        }
    ]
}
```

This was entirely intentional, as described below.

# 2. Explore Logic
Transaction logic - multi-table derivations, constraints,
and actions such as sending mail or messages - is a significant
aspect of any database oriented system, as much as half.

> Logic is the iceberg under the surface of the API.

## Cocktail Napkin Explosion

It is striking how a small "cocktail napkin specification" can balloon into
hundreds of lines of code:

<figure><img src="https://github.com/valhuber/LogicBank/raw/main/images/overview/rules-vs-code.png"></figure>

> Implementing logic by conventional *procedural code is slow, error prone, and painful to maintain.*

## Logic is _Declarative_

Api Logic Server dramatically improves conciseness, quality and maintainability, by introducing a signicant innovation for addressing transaction logic: **Logic Bank**.  This approach is based on two things:

* **Rules - 40X** more concise (see the [contrast here](https://github.com/valhuber/LogicBank/wiki/by-code))
using a declarative, spreadsheet-like paradigm, and

* **Python - control and extensibility,** using standard tools and techniques

**Note:** in this tutorial, the API and Web App were created strictly from the data model.
The logic, however, is injected so you can explore it.

Rules are _not_ simply procedural event handlers.  They are spreadsheet-like expressions for
constraints and derivations.  The "cocktail napkin spec" above is implemented
in ```logic/logic_bank.py```, by declaring just 5 rules that are simply rigorous expressions
of the spec:

<figure><img src="https://github.com/valhuber/ApiLogicServer/raw/main/images/tutorial/declare-logic.png"></figure>

This table summarizes the **key declarative / procedural differences:**

| Characteristic | Procedural   | Declarative  | Why It Matters |
| :-------------: |:-------------:| :-----:| :-----:|
| Reuse     | Not Automatic | Automatic - all Use Cases | 40X Code Reduction |
| Invocation | Passive - only if called | Active - call not required | Quality |
| Ordering | Manual | Automatic | Agile Maintenance |
| Optimizations      | Manual     |   Automatic | Agile Design |

Unlike code, you do not call the rules directly.
The rules engine in Logic Bank listens for SQLAlchemy ```before_flush``` events,
and applies the applicable rules in an order that reflects their dependencies.  For more
information on rules, see the [Logic Bank Overview](https://github.com/valhuber/LogicBank/wiki).

The constraint rule (line 52) is what caused the update error response above -
the ```Balance``` exceeded the altered ```CreditLimit```.  The transaction is rolled back.

## _Automatic_ Re-use
Let's examine the re-use aspect.  In traditional procedural coding, re-use is generally achieved with significant design work.  But in this declarative approach, logic is _automatically **re-used**_ for all transactions.  That has a significant impact on conciseness, and quality.

Let's see how our logic processes a PATCH to an Order Detail:
```
curl -X PATCH "http://localhost:5000/OrderDetail/2156/" \
  -H  "accept: application/vnd.api+json" \
  -H  "Content-Type: application/json" \
  -d '
{
  "data":{
    "attributes": {"Quantity": 10},
    "type": "OrderDetail",
    "id": "2156"  
  }
}'
```

The rule engine generates a
[*log*](https://github.com/valhuber/LogicBank/wiki#debugging-standard-debugger-logic-logging)
showing each rule that fires, the state of the row, with indenting to show rule chaining:

1. Choose the Folder icon on the toolbar to the left
2. Open the file `ApiLogicServerTutorial/Log - Patch Order Detail.txt`
3. From the view menu, disable Word Wrap


This multi-table rollup transaction was automated with the following rules:
* recompute the Order Detail ```Amount``` per the _formula_ rule (line 60)
* adjust the Order ```AmountTotal``` (line 58)
* adjust the Customer ```Balance``` (line 55)
* and check the ```CreditLimit```

> **Key Takeaway:** Logic is 40X more concise, higher quality, and easier to maintain

## Customize with Python

**Alert:** fails per issue 10.  Under investigation.

A common pattern is auditing - if certain attributes are changed, copy the altered row to an audit table.  Let's alter an Employees salary:
```
curl -X PATCH "http://localhost:5000/Employee/1/"\
  -H  "accept: application/vnd.api+json"\
  -H  "Content-Type: application/json"\
  -d "{  \"data\": {\
        \"attributes\": {\
            \"Salary\": \"2000000\"    },\
  \"type\": \"Employee\",\
  \"id\": \"1\"  }}"
```

This is implemented in ```logic/rules_bank.py```:
```python
    def audit_by_event(row: models.Employee, old_row: models.Employee, logic_row: LogicRow):
        tedious = False  # tedious code to repeat for every audited class
        if tedious:      # see instead the following rule extension - nw_copy_row
            if logic_row.are_attributes_changed([models.Employee.Salary, models.Employee.Title]):
                copy_to_logic_row = logic_row.new_logic_row(models.EmployeeAudit)
                copy_to_logic_row.link(to_parent=logic_row)
                copy_to_logic_row.set_same_named_attributes(logic_row)
                copy_to_logic_row.insert(reason="Manual Copy " + copy_to_logic_row.name)  # => rules...
                # logic_row.log("audit_by_event (Manual Copy) complete")

    Rule.commit_row_event(on_class=models.Employee, calling=audit_by_event)

    RuleExtension.copy_row(copy_from=models.Employee,
                           copy_to=models.EmployeeAudit,
                           copy_when=lambda logic_row:
                              logic_row.are_attributes_changed([models.Employee.Salary, models.Employee.Title]))
```
This illustrates 2 aspects of customization:
1. The ```audit_by_event``` approach illustrates that you declare a rule that invokes Python code.  Observe that it is not active, per the ```tedious``` flag
2. The alternative ```RuleExtension.copy_row``` shows a [reusable approach](https://github.com/valhuber/LogicBank/wiki/Rule-Extensibility#3-extended-rules)

> **Key Takeaway:** ApiLogicServer spreadsheet-like logic is 40X more concise, and extensible with standard Python.

# 3. Explore the Basic Web App

UI development takes time.  That's a problem since
* Such effort may not be warranted for admin "back office" screens,
and
  
* [Agile approaches](https://agilemanifesto.org) depend on getting _working
software_ soon, to drive _collaboration and iteration_.

ApiLogicServer CLI therefore creates working software _now:_
multi-page, multi-table applications as shown below:

1. **Multi-page:** apps include 1 page per table

2. **Multi-table:** pages include ``related_views`` for each related child table, and join in parent data

3. **Favorite fields first:** first-displayed field is "name", or `contains` "name" (configurable)

4. **Predictive joins:** favorite field of each parent is shown (product *name* - not product *id*)

5. **Ids last:** such boring fields are not shown on lists, and at the end on other pages

<figure><img src="https://raw.githubusercontent.com/valhuber/fab-quick-start/master/images/generated-page.png"></figure>

To run the application:
```
cd ApiLogicServer/api_logic_server
python ui/basic_web_app/run.py
```

> Before running, you must [Create Admin Data](Working-with-Flask-AppBuilder) for Flask App Builder (except for Northwind, which is pre-created).

## Customize `views.py`
Around line 100, alter these lines in `ui/basic_web_app/app/views.py`for a better **related_views**:
```

class CustomerModelView(ModelView):
   datamodel = SQLAInterface(Customer)
   list_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City"]
   show_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   edit_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   add_columns = ["CompanyName", "ContactName", "ContactTitle", "Address", "City", "Region", "PostalCode", "Country", "Phone", "Fax", "Balance", "CreditLimit", "OrderCount", "Id", "UnpaidOrderCount"]
   related_views = [OrderModelView]

appbuilder.add_view(
      CustomerModelView, "Customer List", icon="fa-folder-open-o", category="Menu")
```

> Flask AppBuilder has a wide range of capabilities, including charts.  For more information, see [Flask AppBuilder](https://flask-appbuilder.readthedocs.io/en/latest/).


> **Key Takeaway:** ApiLogicServer multi-page, multi-table apps provide instant back-office admin and agile prototyping.

# ApiLogicServer Tutorial - Wrap-up

You have built and executed an **ApiLogicServer**, and explored its 3 key elements:

| Element | Key Takeaway   | Why It Matters  |
| :-------------- |:--------------| :------| 
| JSON:**API**  | Created **instantly**, *customizable* | Unblock UI Development |
| **Logic**| Spreadsheet-like rules automate the cocktail napkin spec | **40X more concise**, *customizable* with Python  |
| Basic **Web App** | Multi-page, multi-table - created **instantly**, customizable  | Back-office admin apps, agile prototyping |

In [1]:
!ApiLogicServer version




API Logic Server CLI 01.04.05 here

	Installed at /Users/val/Desktop/jupyter/venv/lib/python3.9/site-packages/api_logic_server_cli/cli.py

Recent Changes:
	02/08/2021 - 01.04.05: add employee audit foreign key in nw.sqlite
	02/07/2021 - 01.04.04: fix default project name
	02/07/2021 - 01.04.03: db_url default (for Jupyter)
	02/07/2021 - 01.04.02: Internal Renaming
	02/06/2021 - 01.04.00: Fix constraint reporting, get related (issues 7,8)
	02/01/2021 - 01.03.00: Fix logic logging, nw rules
	01/31/2021 - 01.03.00: Resolve n:m relationships (revised models.py)
	01/29/2021 - 01.02.04: Minor cleanup
	01/29/2021 - 01.02.03: Flask AppBuilder fixes - Admin setup, class vs table names (wip)
	01/28/2021 - 01.02.02: Command line cleanup
	01/27/2021 - 01.02.00: Host option, from_git defaults to local directory, hello world example, nw rules pre-created
	01/25/2021 - 01.01.01: MySQL fixes

